In [1]:
import pandas as pd
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
load_dotenv()
import os
import openai

In [2]:
# Get the openai api keys
# Get the OpenAI API key and org key from the .env file
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG")

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (ChatPromptTemplate, HumanMessagePromptTemplate)

human_message_prompt = HumanMessagePromptTemplate(prompt = PromptTemplate(input_variables=["liquor", "cocktail_type", "cuisine", "theme", "spirits_and_amounts"],\
                         template = "Given the following parameters: the name of the liquor {liquor} I am trying to use up, the {spirits_and_amounts} that I have in my inventory,\
                              the type of cocktail {cocktail_type}, the type of cuisine {cuisine} I am looking to pair it with, and the theme {theme},\
                                please help me come up with a creative cocktail with a fun and creative name that doesn't\
                                necessarily include the name of the spirit or the theme.  If you can use the spirits in my inventory in spirits_and_amounts that would be great,\
                                      but if not feel free to use any spirits you think would make a good cocktail with my given parameters  Please be as specific as possible with your instructions.\
                                    \n\nRecipe Name: \n\nIngredients: \n\nInstructions: \n\n")
)

chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(model_name = "gpt-3.5-turbo", max_tokens = 1500, temperature = 0.7, top_p = 0.9, frequency_penalty = 0.5, presence_penalty = 0.5,\
            openai_api_key = openai.api_key, openai_organization =  openai.organization)

In [4]:
from langchain.chains import LLMChain
chain = LLMChain(llm = chat, prompt = chat_prompt_template)


In [6]:
spirits_list = [
    'Absinthe',
    'Agave Nectar',
    'Akvavit',
    'Almond Liqueur',
    'Amaretto',
    'Aperol',
    'Apple Brandy',
    'Applejack',
    'Apricot Brandy',
    'Averna',
    'Bailey\'s Irish Cream',
    'Bénédictine',
    'Bitters',
    'Black Sambuca',
    'Blue Curaçao',
    'Bourbon',
    'Brandy',
    'Campari',
    'Chambord',
    'Cherry Brandy',
    'Cider',
    'Cinnamon Schnapps',
    'Cocchi Americano',
    'Cognac',
    'Cointreau',
    'Crème de Cacao',
    'Crème de Cassis',
    'Crème de Menthe',
    'Crème de Violette',
    'Curacao',
    'Drambuie',
    'Dry Vermouth',
    'Elderflower Liqueur',
    'Falernum',
    'Fernet Branca',
    'Galliano',
    'Genever',
    'Gin',
    'Gold Rum',
    'Goldschläger',
    'Grand Marnier',
    'Grapefruit Liqueur',
    'Green Chartreuse',
    'Grenadine',
    'Hazelnut Liqueur',
    'Hpnotiq',
    'Irish Cream',
    'Jägermeister',
    'Kahlua',
    'Lemon Juice',
    'Lemoncello',
    'Lillet',
    'Lime Juice',
    'Maraschino Liqueur',
    'Mezcal',
    'Midori',
    'Mint Leaves',
    'Nutmeg',
    'Orange Bitters',
    'Orange Curacao',
    'Orange Flower Water',
    'Orange Juice',
    'Orgeat Syrup',
    'Overproof Rum',
    'Peach Bitters',
    'Peach Liqueur',
    'Peppermint Schnapps',
    'Pernod',
    'Peychaud\'s Bitters',
    'Pimm\'s No. 1',
    'Pineapple Juice',
    'Port',
    'Prosecco',
    'Pumpkin Liqueur',
    'Raspberry Liqueur',
    'Red Wine',
    'Rhum Agricole',
    'Rose Water',
    'Rum',
    'Rye Whiskey',
    'Sake',
    'Sambuca',
    'Scotch',
    'Sherry',
    'Simple Syrup',
    'Sloe Gin',
    'Smoked Paprika',
    'Southern Comfort',
    'Sparkling Wine',
    'Spiced Rum',
    'St. Germain',
    'Strawberry Liqueur',
    'Sweet and Sour Mix',
    'Sweet Vermouth',
    'Tabasco Sauce',
    'Tequila',
    'Tomato Juice',
    'Triple Sec',
    'Vanilla Liqueur',
    'Vanilla Syrup',
    'Vermouth',
    'Vodka',
    'Whiskey',
    'White Rum',
    'White Wine',
    'Yellow Chartreuse'
]



In [7]:
# Create a pandas dataframe with the list of spirits
df = pd.DataFrame(spirits_list, columns = ["liquor"])

In [8]:
import numpy as np
# Add a column to the dataframe called "amount" and generate a random number between 1 and 1000 for each row
df["amount"] = np.random.randint(1, 1000, df.shape[0])


In [9]:
# Change the name of the "amount" columns to "amount_in_oz"
df.rename(columns = {"amount": "amount_in_oz"}, inplace = True)
df

,liquor,amount_in_oz
0,Absinthe,991
1,Agave Nectar,92
2,Akvavit,327
3,Almond Liqueur,876
4,Amaretto,902
...,...,...
101,Vodka,893
102,Whiskey,292
103,White Rum,631
104,White Wine,352


In [10]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", max_tokens = 750, temperature = 0.7, top_p = 0.9, frequency_penalty = 0.5, presence_penalty = 0.5,\
            openai_api_key = openai.api_key, openai_organization =  openai.organization)


In [13]:
# Export the dataframe to a csv file
df.to_csv("liquor_amounts.csv", index = False)

from langchain.agents import create_csv_agent

In [14]:
# Create the agent using the csv file
agent = create_csv_agent(llm, "liquor_amounts.csv", verbose = True)


In [15]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [18]:
class SpiritsList(BaseModel):
    spirits: List[str] = Field(description = "A list of spirits")
    amounts: List[int] = Field(description = "A list of amounts in oz")

    
spirits_query = "Return a list of spirits in the .csv file and their amounts in ml.  You will need to convert\
                the amounts in oz to ml."

parser = PydanticOutputParser(pydantic_object=SpiritsList, query=spirits_query)

spirits_and_amounts = agent.run(input = "Return a list of spirits and their amounts in ml", parser=parser)


     



> Entering new AgentExecutor chain...
Thought: We need to select the 'liquor' and 'amount_in_oz' columns, convert the amount to ml, and return the result as a list.
Action: python_repl_ast
Action Input:
```
df['amount_in_ml'] = df['amount_in_oz'] * 29.5735
result = df[['liquor', 'amount_in_ml']].values.tolist()
```
Observation: 
Thought:We have created a new column 'amount_in_ml' by multiplying the 'amount_in_oz' column by 29.5735 to convert it to ml. We then selected the 'liquor' and 'amount_in_ml' columns using double brackets and converted the result to a list using the .values.tolist() method.
Action: python_repl_ast
Action Input:
```
result
```
Observation: [['Absinthe', 29307.338499999998], ['Agave Nectar', 2720.7619999999997], ['Akvavit', 9670.5345], ['Almond Liqueur', 25906.386], ['Amaretto', 26675.297], ['Aperol', 14757.1765], ['Apple Brandy', 22327.9925], ['Applejack', 8428.4475], ['Apricot Brandy', 3371.379], ['Averna', 13071.487], ["Bailey's Irish Cream", 20524.009], ['Bé

In [15]:
spirits_and_amounts

'[(\'Absinthe\', 537), (\'Agave Nectar\', 941), (\'Akvavit\', 940), (\'Almond Liqueur\', 653), (\'Amaretto\', 741), (\'Aperol\', 751), (\'Apple Brandy\', 872), (\'Applejack\', 903), (\'Apricot Brandy\', 164), (\'Averna\', 372), ("Bailey\'s Irish Cream", 913...)]'

In [19]:
print(chain.run(liquor = "rum", cocktail_type = "craft", cuisine = "cuban", theme = "summer", spirits_and_amounts = spirits_and_amounts))

Recipe Name: Havana Sunset

Ingredients:
- 1 oz rum
- 1 oz apricot brandy
- 1/2 oz amaretto
- 1/2 oz agave nectar
- 1/4 oz absinthe
- 1 oz orange juice
- 1/2 oz lime juice
- Splash of grenadine

Instructions:
1. In a cocktail shaker filled with ice, combine the rum, apricot brandy, amaretto, agave nectar, and absinthe.
2. Shake well and strain into a tall glass filled with ice.
3. Add the orange juice and lime juice to the glass and stir.
4. Slowly pour in a splash of grenadine to create a sunset effect.
5. Garnish with a slice of orange or lime.

Enjoy your tropical Havana Sunset cocktail that pairs perfectly with Cuban cuisine!
